In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [6]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/10월/나무문화_10월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

386


,조사,조사빈도
0,을,575984
1,의,471313
2,이,417776
3,에,404215
4,를,275793
...,...,...
383,까지만은,1
384,에게서부터,1
385,로구먼,1
386,으로서나,1


In [7]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_10월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,방문한지 4달이 조금 안된시점에 포스팅을 하게되었다,0,"['방문', '한지', '4', '달이', '조금', '안된', '시점', '에',...","[('방문', 'Noun'), ('한지', 'Noun'), ('4', 'Number..."
1,사실은 밀려있는 그릿비에게 밀려서 잊고 있었던 코랄라니 카페 공사하는모습은 진작에 ...,0,"['사실', '은', '밀려있는', '그릿비', '에게', '밀려서', '잊고', ...","[('사실', 'Noun'), ('은', 'Josa'), ('밀리다', 'Verb'..."
2,대형카페의 규모는 도대체 얼마나 더 커질예정인지 참 가늠이 안되는구나 라는 생각을 ...,0,"['대형', '카페', '의', '규모', '는', '도대체', '얼마나', '더'...","[('대형', 'Noun'), ('카페', 'Noun'), ('의', 'Josa')..."
3,근래 생긴 기장에 속해있는 카페중에 주차장이 가장 잘되어있다고 생각 수용인원에대한 ...,0,"['근래', '생긴', '기장', '에', '속', '해있는', '카페', '중',...","[('근래', 'Noun'), ('생기다', 'Verb'), ('기장', 'Noun..."
4,오픈 일자에 방문했는데도 미리 걸려있던 현수막 덕분인지 주차장을 들어가는데 한참을 ...,0,"['오픈', '일자', '에', '방문', '했는데도', '미리', '걸려있던', ...","[('오픈', 'Noun'), ('일자', 'Noun'), ('에', 'Josa')..."
...,...,...,...,...
705292,일 년 중 한 달을 아주 진하게 찾아오는 우울에 적셔지고 있노라면 blog nave...,13119,"['일', '년', '중', '한', '달', '을', '아주', '진하게', '찾...","[('일', 'Noun'), ('년', 'Noun'), ('중', 'Noun'), ..."
705293,1984 동물농장 을 읽을 때도 느꼈다 옛날 사람이 자신의 시대를 신랄하게 비판한 ...,13119,"['1984', '동물농장', '을', '읽을', '때', '도', '느꼈다', '...","[('1984', 'Number'), ('동물농장', 'Noun'), ('을', '..."
705294,나는 나 자신이 되면 된다 인간이라면 으레 경험하는 감정을 유순한 마음으로 받아들이...,13119,"['나', '는', '나', '자신', '이', '되면', '된다', '인간', '...","[('나', 'Noun'), ('는', 'Josa'), ('나', 'Noun'), ..."
705295,끼야홍 독후감을 좀 더 고심해서 적어야겠어,13119,"['끼', '야홍', '독후감', '을', '좀', '더', '고심', '해서', ...","[('끼', 'Noun'), ('야홍', 'Noun'), ('독후감', 'Noun'..."


In [10]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,방문한지 4달이 조금 안된시점에 포스팅을 하게되었다,0,"['방문', '한지', '4', '달이', '조금', '안된', '시점', '에',...","[('방문', 'Noun'), ('한지', 'Noun'), ('4', 'Number...",방문 한지 4 달이 조금 안된 시점 에 포스팅 을 하게 되었다
1,사실은 밀려있는 그릿비에게 밀려서 잊고 있었던 코랄라니 카페 공사하는모습은 진작에 ...,0,"['사실', '은', '밀려있는', '그릿비', '에게', '밀려서', '잊고', ...","[('사실', 'Noun'), ('은', 'Josa'), ('밀리다', 'Verb'...",사실 은 밀려있는 그릿비 에게 밀려서 잊고 있었던 코랄 라니 카페 공사 하는 모습 ...
2,대형카페의 규모는 도대체 얼마나 더 커질예정인지 참 가늠이 안되는구나 라는 생각을 ...,0,"['대형', '카페', '의', '규모', '는', '도대체', '얼마나', '더'...","[('대형', 'Noun'), ('카페', 'Noun'), ('의', 'Josa')...",대형 카페 의 규모 는 도대체 얼마나 더 커질 예정 인지 참 가늠 이 안되는구나 라...
3,근래 생긴 기장에 속해있는 카페중에 주차장이 가장 잘되어있다고 생각 수용인원에대한 ...,0,"['근래', '생긴', '기장', '에', '속', '해있는', '카페', '중',...","[('근래', 'Noun'), ('생기다', 'Verb'), ('기장', 'Noun...",근래 생긴 기장 에 속 해있는 카페 중 에 주차장 이 가장 잘 되어있다고 생각 수 ...
4,오픈 일자에 방문했는데도 미리 걸려있던 현수막 덕분인지 주차장을 들어가는데 한참을 ...,0,"['오픈', '일자', '에', '방문', '했는데도', '미리', '걸려있던', ...","[('오픈', 'Noun'), ('일자', 'Noun'), ('에', 'Josa')...",오픈 일자 에 방문 했는데도 미리 걸려있던 현수막 덕분 인지 주차장 을 들어가는데 ...
...,...,...,...,...,...
705292,일 년 중 한 달을 아주 진하게 찾아오는 우울에 적셔지고 있노라면 blog nave...,13119,"['일', '년', '중', '한', '달', '을', '아주', '진하게', '찾...","[('일', 'Noun'), ('년', 'Noun'), ('중', 'Noun'), ...",일 년 중 한 달 을 아주 진하게 찾아오는 우울 에 적셔지고 있노라면 blog na...
705293,1984 동물농장 을 읽을 때도 느꼈다 옛날 사람이 자신의 시대를 신랄하게 비판한 ...,13119,"['1984', '동물농장', '을', '읽을', '때', '도', '느꼈다', '...","[('1984', 'Number'), ('동물농장', 'Noun'), ('을', '...",1984 동물농장 을 읽을 때 도 느꼈다 옛날 사람 이 자신 의 시대 를 신랄하게 ...
705294,나는 나 자신이 되면 된다 인간이라면 으레 경험하는 감정을 유순한 마음으로 받아들이...,13119,"['나', '는', '나', '자신', '이', '되면', '된다', '인간', '...","[('나', 'Noun'), ('는', 'Josa'), ('나', 'Noun'), ...",나 는 나 자신 이 되면 된다 인간 이 라면 으레 경험 하는 감정 을 유순한 마음 ...
705295,끼야홍 독후감을 좀 더 고심해서 적어야겠어,13119,"['끼', '야홍', '독후감', '을', '좀', '더', '고심', '해서', ...","[('끼', 'Noun'), ('야홍', 'Noun'), ('독후감', 'Noun'...",끼 야홍 독후감 을 좀 더 고심 해서 적어야겠어


In [11]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,방문한지 4달이 조금 안된시점에 포스팅을 하게되었다,0,"['방문', '한지', '4', '달이', '조금', '안된', '시점', '에',...","[('방문', 'Noun'), ('한지', 'Noun'), ('4', 'Number...",방문 한지 4 달이 조금 안된 시점 에 포스팅 을 하게 되었다,방문 한지 4 달이 조금 안된 시점 에 포스팅 을 하게 되었다 사실 은 밀려있는 그...
1,사실은 밀려있는 그릿비에게 밀려서 잊고 있었던 코랄라니 카페 공사하는모습은 진작에 ...,0,"['사실', '은', '밀려있는', '그릿비', '에게', '밀려서', '잊고', ...","[('사실', 'Noun'), ('은', 'Josa'), ('밀리다', 'Verb'...",사실 은 밀려있는 그릿비 에게 밀려서 잊고 있었던 코랄 라니 카페 공사 하는 모습 ...,방문 한지 4 달이 조금 안된 시점 에 포스팅 을 하게 되었다 사실 은 밀려있는 그...
2,대형카페의 규모는 도대체 얼마나 더 커질예정인지 참 가늠이 안되는구나 라는 생각을 ...,0,"['대형', '카페', '의', '규모', '는', '도대체', '얼마나', '더'...","[('대형', 'Noun'), ('카페', 'Noun'), ('의', 'Josa')...",대형 카페 의 규모 는 도대체 얼마나 더 커질 예정 인지 참 가늠 이 안되는구나 라...,방문 한지 4 달이 조금 안된 시점 에 포스팅 을 하게 되었다 사실 은 밀려있는 그...
3,근래 생긴 기장에 속해있는 카페중에 주차장이 가장 잘되어있다고 생각 수용인원에대한 ...,0,"['근래', '생긴', '기장', '에', '속', '해있는', '카페', '중',...","[('근래', 'Noun'), ('생기다', 'Verb'), ('기장', 'Noun...",근래 생긴 기장 에 속 해있는 카페 중 에 주차장 이 가장 잘 되어있다고 생각 수 ...,방문 한지 4 달이 조금 안된 시점 에 포스팅 을 하게 되었다 사실 은 밀려있는 그...
4,오픈 일자에 방문했는데도 미리 걸려있던 현수막 덕분인지 주차장을 들어가는데 한참을 ...,0,"['오픈', '일자', '에', '방문', '했는데도', '미리', '걸려있던', ...","[('오픈', 'Noun'), ('일자', 'Noun'), ('에', 'Josa')...",오픈 일자 에 방문 했는데도 미리 걸려있던 현수막 덕분 인지 주차장 을 들어가는데 ...,방문 한지 4 달이 조금 안된 시점 에 포스팅 을 하게 되었다 사실 은 밀려있는 그...
...,...,...,...,...,...,...
705292,일 년 중 한 달을 아주 진하게 찾아오는 우울에 적셔지고 있노라면 blog nave...,13119,"['일', '년', '중', '한', '달', '을', '아주', '진하게', '찾...","[('일', 'Noun'), ('년', 'Noun'), ('중', 'Noun'), ...",일 년 중 한 달 을 아주 진하게 찾아오는 우울 에 적셔지고 있노라면 blog na...,올해 독서 목표 는 내 가 정 한 필독 서 완독 하기 연 50 권 읽기 내 가 정 ...
705293,1984 동물농장 을 읽을 때도 느꼈다 옛날 사람이 자신의 시대를 신랄하게 비판한 ...,13119,"['1984', '동물농장', '을', '읽을', '때', '도', '느꼈다', '...","[('1984', 'Number'), ('동물농장', 'Noun'), ('을', '...",1984 동물농장 을 읽을 때 도 느꼈다 옛날 사람 이 자신 의 시대 를 신랄하게 ...,올해 독서 목표 는 내 가 정 한 필독 서 완독 하기 연 50 권 읽기 내 가 정 ...
705294,나는 나 자신이 되면 된다 인간이라면 으레 경험하는 감정을 유순한 마음으로 받아들이...,13119,"['나', '는', '나', '자신', '이', '되면', '된다', '인간', '...","[('나', 'Noun'), ('는', 'Josa'), ('나', 'Noun'), ...",나 는 나 자신 이 되면 된다 인간 이 라면 으레 경험 하는 감정 을 유순한 마음 ...,올해 독서 목표 는 내 가 정 한 필독 서 완독 하기 연 50 권 읽기 내 가 정 ...
705295,끼야홍 독후감을 좀 더 고심해서 적어야겠어,13119,"['끼', '야홍', '독후감', '을', '좀', '더', '고심', '해서', ...","[('끼', 'Noun'), ('야홍', 'Noun'), ('독후감', 'Noun'...",끼 야홍 독후감 을 좀 더 고심 해서 적어야겠어,올해 독서 목표 는 내 가 정 한 필독 서 완독 하기 연 50 권 읽기 내 가 정 ...


In [12]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,방문 한지 4 달이 조금 안된 시점 에 포스팅 을 하게 되었다 사실 은 밀려있는 그...
1,1,안녕하세요 벨라 예요 여러분 설날 이네 요 새해 가 정말 시작 됐군요 광광 ㅠ 마지...
2,2,향기 와 함께 하는 전시 가 있다는 말 에 냉큼 가보기로 했다 파주 헤 이리 마을 ...
3,3,나전 합의 매력 그대로 자개 스마트 톡 금속 배지 어린 시절 에 용돈 을 받으면 어...
4,4,저희 시댁 은 차례 나 명절 음식 따로 없이 명절 에 모이 면 회 를 떠다가 초밥 ...
...,...,...
13115,13115,전주 체련공원 과 동물원 사이 건지산 에 베짱이 숲 이라는 숲속 놀이터 가 있는데용...
13116,13116,안녕하세요 깜찍 이 헐크 입니다 자 오늘 은 중국 의 뱀파이어 강시 에 대해 포스팅...
13117,13117,주부 라니 입니다 새해 복 많이 받으세요 소원 은 잘 빌고 오셨나요 올 해도 건강하...
13118,13118,blog 조 선제 4 대 세종 177 개 의 글 조 선제 4 대 세종 목록 닫기 조...


In [13]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [14]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [15]:
tdm.shape

(13120, 393790)

In [16]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02816018, 0.02156298, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.01751953, 0.02683031, ..., 0.        , 0.        ,
        0.        ],
       [0.00390889, 0.01848562, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [17]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,393780,393781,393782,393783,393784,393785,393786,393787,393788,393789
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.028160,0.021563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.033795,0.031964,0.032634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,0.000000,0.124318,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13116,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13117,0.000000,0.017520,0.026830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13118,0.003909,0.018486,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [19]:
tfidf_DF

,0,00,000,0001,000100,0001139070,00012,0001395107,0001398019,0001399289,...,힛데겔,힛송,힛타이트,힛트,힜,힝,힝구,힝규,힝힝,힣
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.028160,0.021563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.033795,0.031964,0.032634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13115,0.000000,0.124318,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13116,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13117,0.000000,0.017520,0.026830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13118,0.003909,0.018486,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/10월/나무문화_10월_TF-IDF행렬(원본).txt", index=False, encoding='UTF-8')

In [ ]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

In [49]:
# 한글로만 이루어진 TF-IDF 행렬도 txt로 저장해줌.
korean_df.to_csv("../dataset/2022/본문_TF-IDF/10월/나무문화_10월_TF-IDF행렬(원본).txt", index=False, encoding='UTF-8')

In [50]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,51.032181
1,00,50.503403
2,000,19.454386
3,0001,0.049739
4,0001076717,0.034615
...,...,...
134284,힙니,0.062174
134285,힙스터,0.058074
134286,힙합,0.405733
134287,힝,0.086247


In [51]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/10월/나무문화_10월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')